Analyze Dataset from overnight runs

In [5]:
import pandas as pd

data = pd.DataFrame([[60424, 4096],
[76524, 4096],
[37008, 2048],
[65964, 4096],
[36456, 2048],
[3924, 256],
[35092, 2048],
[123296, 8192],
[79392, 4096],
[71724, 4096],
[36424, 2048],
[36116, 2048],
[18344, 1024],
[77152, 4096],
[57040, 4096],
[15072, 1024],
[76420, 4096],
[113608, 8192],
[61344, 4096],
[79848, 4096],
[52092, 4096],
[24988, 2048],
[80320, 4096],
[26460, 2048],
[28348, 2048],
[24016, 1024],
[111924, 8192],
[63596, 4096],
[60956, 4096],
[32592, 2048],
[36212, 2048],
[57040, 4096],
[36524, 2048],
[32348, 2048],
[16132, 1024],
[67820, 4096],
[80444, 4096],
[15840, 1024],
[36460, 2048],
[78552, 4096],
[72040, 4096],
[37204, 2048],
[59496, 4096],
[70104, 4096],
[58016, 4096],
[32656, 2048],
[27536, 2048],
[81744, 4096],
[36044, 2048],
[34944, 2048],
[72424, 4096],
[34872, 2048],
[47728, 4096],
[34840, 2048],
[37152, 2048],
[60636, 4096],
[76804, 4096],
[36156, 2048],
[78056, 4096],
[111888, 8192],
[76784, 4096],
[34328, 2048],
[50228, 4096],
[36848, 2048],
[32364, 2048],
[31480, 2048]],
columns=['score', 'max_tile'])

num_over_1024 = len(data[data['max_tile']>=1024])
num_over_2048 = len(data[data['max_tile']>=2048])
num_over_4096 = len(data[data['max_tile']>=4096])
num_over_8196 = len(data[data['max_tile']>=8192])

print(f'Number of games over 1024: {num_over_1024/len(data)}')
print(f'Number of games over 2048: {num_over_2048/len(data)}')
print(f'Number of games over 4096: {num_over_4096/len(data)}')
print(f'Number of games over 8192: {num_over_8196/len(data)}')
print(len(data))

Number of games over 1024: 0.9848484848484849
Number of games over 2048: 0.9090909090909091
Number of games over 4096: 0.5151515151515151
Number of games over 8192: 0.06060606060606061
66


In [6]:
from Reporter import LightConcurrentReporter
from MarkovDPAI import *
import numpy as np

def perform_trial(parameters):
    path_penalty = parameters.get('path_penalty')
    position_penalty = parameters.get('position_penalty')
    loss_penalty = parameters.get('loss_penalty')
    score_factor = parameters.get('score_factor')

    ai_input=MarkovDPAI(ExpectiMax6(path_pen=path_penalty, pos_pen=position_penalty, loss_penalty=loss_penalty, score_factor=score_factor))
    r = LightConcurrentReporter(ai_input, 50)
    report = pd.DataFrame(r.generate_report(), columns=['score','max_tile'])
    print(np.mean(report['score']), len(report[report['max_tile']>=2048])/50.0)
    return len(report[report['max_tile']>=2048])/50.0

In [9]:
from ax.service.ax_client import AxClient
from ax.modelbridge.registry import Models
from ax.modelbridge.generation_strategy import GenerationStrategy, GenerationStep

generation_strategy = GenerationStrategy(
    steps=[
        GenerationStep(model=Models.SOBOL, num_trials=10),
        GenerationStep(model=Models.GPEI, num_trials=-1),   # GPEI for the rest of the trials
    ]
)
ax_client = AxClient(generation_strategy=generation_strategy)
ax_client.create_experiment(name='optimize Params',
                            parameters=[
                                {'name': 'path_penalty',
                                    'type': 'range',
                                    'bounds': [0.0, 9.0]},
                                {'name': 'position_penalty',
                                    'type': 'range',
                                    'bounds': [0.0, 9.0]},
                                {'name': 'loss_penalty',
                                    'type': 'range',
                                    'bounds': [0.0, 10.0]},
                                {'name': 'score_factor',
                                    'type': 'range',
                                    'bounds': [0.0, 5.0]},
                            ],
                            status_quo={'path_penalty':4.266638760734974,
                                        'position_penalty':0,
                                        'loss_penalty':8.466460274888114,
                                        'score_factor':2.0})

for i in range(10):
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=perform_trial(parameters))


for i in range(10):
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=perform_trial(parameters))

best_parameters, metrics = ax_client.get_best_parameters()
print(f'The best parameters are: {best_parameters}')
print(f'with score {metrics}')

[INFO 10-20 09:57:19] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 10-20 09:57:19] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter path_penalty. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 10-20 09:57:19] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter position_penalty. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 10-20 09:57:19] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter loss_penalty. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.

34522.4 0.7


[INFO 10-20 10:09:47] ax.service.ax_client: Completed trial 1 with data: {'objective': (0.24, None)}.
/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/modelbridge/cross_validation.py:463: UserWarning: Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.
  warn("Encountered exception in computing model fit quality: " + str(e))
[INFO 10-20 10:09:47] ax.service.ax_client: Generated new trial 2 with parameters {'path_penalty': 4.209805, 'position_penalty': 2.076866, 'loss_penalty': 3.176201, 'score_factor': 0.485624} using model Sobol.


15397.04 0.24


[INFO 10-20 10:13:54] ax.service.ax_client: Completed trial 2 with data: {'objective': (0.4, None)}.
/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/modelbridge/cross_validation.py:463: UserWarning: Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.
  warn("Encountered exception in computing model fit quality: " + str(e))
[INFO 10-20 10:13:54] ax.service.ax_client: Generated new trial 3 with parameters {'path_penalty': 4.716465, 'position_penalty': 4.704588, 'loss_penalty': 9.623575, 'score_factor': 3.064852} using model Sobol.


...

28358.16 0.56


[INFO 10-20 11:49:06] ax.service.ax_client: Generated new trial 19 with parameters {'path_penalty': 6.916087, 'position_penalty': 5.063317, 'loss_penalty': 10.0, 'score_factor': 3.542694} using model GPEI.
[INFO 10-20 11:55:30] ax.service.ax_client: Completed trial 19 with data: {'objective': (0.5, None)}.
[WARNING 10-20 11:55:30] ax.modelbridge.base: Status quo status_quo not present in data


24566.4 0.5


/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/utils/stats/model_fit_stats.py:199: RuntimeWarning: divide by zero encountered in divide
  return float(np.mean(1.96 * 2 * se_pred / np.abs(y_obs)))
/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/utils/stats/model_fit_stats.py:229: RuntimeWarning: divide by zero encountered in scalar divide
  return float((np.max(y_obs) - min_y_obs) / min_y_obs)


The best parameters are: {'path_penalty': 7.769070982933044, 'position_penalty': 2.945681869983673, 'loss_penalty': 2.3631465435028076, 'score_factor': 1.9154930114746094}
with score ({'objective': 0.6936248310047561}, {'objective': {'objective': 0.0002636204603058724}})


In [20]:
for i in range(60):
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=perform_trial(parameters))
    
best_parameters, metrics = ax_client.get_best_parameters()
print(f'The best parameters are: {best_parameters}')
print(f'with score {metrics}')

/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/core/data.py:288: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return cls(df=pd.concat(dfs, axis=0, sort=True))
[INFO 10-21 21:21:22] ax.service.ax_client: Generated new trial 161 with parameters {'path_penalty': 7.516301, 'position_penalty': 9.0, 'loss_penalty': 4.504579, 'score_factor': 3.217935} using model GPEI.
[INFO 10-21 21:28:14] ax.service.ax_client: Completed trial 161 with data: {'objective': (0.62, None)}.


27206.24 0.62


/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/core/data.py:288: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return cls(df=pd.concat(dfs, axis=0, sort=True))
[INFO 10-21 21:28:19] ax.service.ax_client: Generated new trial 162 with parameters {'path_penalty': 9.0, 'position_penalty': 0.0, 'loss_penalty': 0.994676, 'score_factor': 4.428855} using model GPEI.
[INFO 10-21 21:35:13] ax.service.ax_client: Completed trial 162 with data: {'objective': (0.56, None)}.


25535.76 0.56


/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/core/data.py:288: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return cls(df=pd.concat(dfs, axis=0, sort=True))
[INFO 10-21 21:35:18] ax.service.ax_client: Generated new trial 163 with parameters {'path_penalty': 9.0, 'position_penalty': 0.0, 'loss_penalty': 4.287498, 'score_factor': 3.534618} using model GPEI.
[INFO 10-21 21:43:49] ax.service.ax_client: Completed trial 163 with data: {'objective': (0.78, None)}.


...

/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/core/data.py:288: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return cls(df=pd.concat(dfs, axis=0, sort=True))
[INFO 10-22 04:13:11] ax.service.ax_client: Generated new trial 219 with parameters {'path_penalty': 7.198084, 'position_penalty': 0.0, 'loss_penalty': 7.517054, 'score_factor': 1.773722} using model GPEI.
[INFO 10-22 04:20:29] ax.service.ax_client: Completed trial 219 with data: {'objective': (0.76, None)}.


33932.0 0.76


/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/core/data.py:288: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return cls(df=pd.concat(dfs, axis=0, sort=True))
[INFO 10-22 04:20:34] ax.service.ax_client: Generated new trial 220 with parameters {'path_penalty': 7.751063, 'position_penalty': 0.0, 'loss_penalty': 8.355485, 'score_factor': 2.899282} using model GPEI.
[INFO 10-22 04:27:12] ax.service.ax_client: Completed trial 220 with data: {'objective': (0.64, None)}.
/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/core/data.py:288: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the 

28406.32 0.64
The best parameters are: {'path_penalty': 3.23005488885216, 'position_penalty': 0.0, 'loss_penalty': 10.0, 'score_factor': 1.01855712728175}
with score ({'objective': 0.723483703188145}, {'objective': {'objective': 0.0003219295932833843}})


/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/utils/stats/model_fit_stats.py:199: RuntimeWarning: divide by zero encountered in divide
  return float(np.mean(1.96 * 2 * se_pred / np.abs(y_obs)))
/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/utils/stats/model_fit_stats.py:229: RuntimeWarning: divide by zero encountered in scalar divide
  return float((np.max(y_obs) - min_y_obs) / min_y_obs)


In [37]:
data =  ax_client.experiment.fetch_data()
print(data.df)
print(data.df.sort_values(by='mean', ascending=False).head(10))

trial_params = ax_client.experiment.trials[165]
print(trial_params)

    arm_name metric_name  mean  sem  trial_index
0        0_0   objective  0.70  NaN            0
1        1_0   objective  0.24  NaN            1
2        2_0   objective  0.40  NaN            2
3        3_0   objective  0.62  NaN            3
4        4_0   objective  0.60  NaN            4
..       ...         ...   ...  ...          ...
215    216_0   objective  0.72  NaN          216
216    217_0   objective  0.74  NaN          217
217    218_0   objective  0.70  NaN          218
218    219_0   objective  0.76  NaN          219
219    220_0   objective  0.64  NaN          220

[220 rows x 5 columns]
    arm_name metric_name  mean  sem  trial_index
185    186_0   objective  0.82  NaN          186
101    102_0   objective  0.82  NaN          102
164    165_0   objective  0.80  NaN          165
132    133_0   objective  0.80  NaN          133
91      92_0   objective  0.80  NaN           92
83      83_0   objective  0.80  NaN           83
122    123_0   objective  0.80  NaN          